# Testing
## Imports

In [1]:
import asyncio
import multiprocessing
import discordBack, discordFront, langDocBack
from Conversation import Conversation, BotMessage, UserMessage, SystemMessage
from collections import defaultdict
from multiprocessing import Pool, current_process
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage
)
import asyncio, logging
from multiprocessing import Process, freeze_support, log_to_stderr

chat = ChatOpenAI(temperature=0)

langDocBack.initSysMsgs()
logging.basicConfig(level=logging.INFO, filename="logsBack.log")
logger = logging.getLogger()

## func definitions

In [2]:
## initializes a patient simulation with id, initial inputMessages (e.g. initial questions from doc) and patient specification 
## defaults to glaucoma patient if no patient specification is passed
## initializes patientSimulations[id]["patientConvo"] with all initial messages and adds first patient response
async def initPatientSimulation(id, inputMessages, patientSpecification = None):
    logger.info("Initializing Patient Simulation with ID "+ str(id)+ " (testing.initPatientSimulation())")
    patientSimulation = defaultdict(dict)
    patientSimulation["id"] = "Patient "+str(id)
    patientSimulation["patientConvo"] = None
    #patientSimulation["patientConvo"] = None

    if not patientSpecification:
        patientSpecification = "Ms. Rodriqguez, a 60 year old smoker with yet undiagnosed glaucoma."
    
    patientSimulation["patientSpecification"] = patientSpecification

    patientSys = """
    You are PatientGPT, designed to simulate patients as realistically as possible for med students and doctors to practice their anamnesis skills.
    Today, you will play the following patient:"""+ patientSimulation["patientSpecification"] +""".
    As the medics are asking you questions, make up realistic symptoms, patient history and do a good job of providing a realistic patient anamnesis experience. This means that you sometimes tell irrelevant information and sometimes don't share information unless explicitly asked.
    Keep in mind that you should only ever simulate the responses of the patient and let the students/doctors ask the questions. 
    The next thing the patient says is:
    """

    patientConvo = Conversation(SystemMessage(patientSys))
    
    for msg in inputMessages:
        patientConvo.addMessage(HumanMessage(content=msg.content))
    
    patientAnswer = chat(patientConvo)

    logger.info("Patient:"+patientAnswer.content+ "(testing.initPatientSimulation())")
    #logger.info("testing.initPatientSimulation(): Patient:"+patientAnswer.content)
    patientConvo.addMessage(patientAnswer)
    patientSimulation["patientConvo"] = patientConvo
    return patientSimulation

In [3]:
## takes patient simulation and appends one patient response to it, returns patientsimulation with patientSimulation["patientConvo"] with appended response
def generatePatientAnswer(patientSimulation):
    ## fetches patientConvo based on id
    
    patientConvo = patientSimulation["patientConvo"]

    # generates patient response based on patientConvo and updates patientConvo
    #logger.info("testing.generatePatientAnswer() - patientConvo:")
    #logger.info(patientConvo)
    patientResponse = chat(patientConvo)
    logger.info("Patient:"+patientResponse.content + "(testing.generatePatientAnswer())")
    patientConvo.addMessage(patientResponse)

    patientSimulation["patientConvo"] = patientConvo
    #logger.info("Patient:"+patientSimulations[id]["patientConvo"][-1].content)
    return patientSimulation

In [4]:

## initializes a demo conversation between a patient simulation and langdoc
async def initDemoConversation():

    # initializes patient simulation id
    testDict = defaultdict(dict)
    user_context=testDict["demoConversation"]
    user_context["user_id"]="demoConversation"
    
    # initializes langdoc api and first doc messages and prints them
    tester=langDocBack.initLangDocAPI(user_context)["docConvo"]
    #logger.info(tester)
    response = Conversation(tester)
    #response.print
    response.flipRoles(False)
    response.addMessage("To get started, could you please provide your age, sex, and nationality?","user")

    #for msg in response:
        #logger.info("Doctor: "+msg.content)

    
    # initializes patient simulation with doc messages and simulation id
    patientSimulation = await initPatientSimulation(user_context["user_id"],response["docConvo"],None)
    patientConvo = patientSimulation["patientConvo"]

    # gets doctor's first answer to patient's intro messages
    docAnswer = Conversation(langDocBack.initPatientConvo(patientConvo.lastMessage().content, user_context)["docConvo"])

    patientConvo.addMessage(UserMessage(docAnswer.lastMessage().content))
    patientSimulation["patientConvo"] = patientConvo
    
    #logger.info("testing.initDemoConversation() - patientConvo after last init:")
    #logger.info(patientSimulations[user_context["user_id"]]["patientConvo"])

    #logger.info("Doctor: "+patientSimulations[user_context["user_id"]]["patientConvo"][-1].content)

    while True:
        patientSimulation = generatePatientAnswer(patientSimulation)
        patientConvo = patientSimulation["patientConvo"]
        docReply = langDocBack.processResponse(patientConvo[-1].content, user_context)
        patientSimulation["patientConvo"].addMessage(UserMessage(docReply["docConvo"].lastMessage().content)) 
       # logger.info("Doctor: "+docReply[-1].content)
        #patientConvo.logger.info("\n\n TESTING COUNTMESSAGE FUNCTION")
        #logger.info("Count:"+str(patientConvo.countMessagesOfType(HumanMessage)))

In [5]:
async def runPatientSimulation(id, vignette):
    try:
        logger = logging.getLogger(str(current_process().pid)) 
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handler = logging.StreamHandler()
        #handler = logging.FileHandler('logfile_{current_process().pid}.log')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        logger.setLevel(logging.INFO)

        testDict = defaultdict(dict)
        user_context=testDict["demoConversation"]
        user_context["user_id"]= id
        

        # initializes langdoc api and first doc messages and prints them, flips roles of generated answers so that patient AI thinks that the doc AI's messages are coming from the user
        tester=langDocBack.initLangDocAPI(user_context, logger)
        #logger.info(tester)
        response = Conversation(tester["docConvo"])
        #response.print
        response.flipRoles(False)
        response.addMessage("To get started, could you please provide your age, sex, and nationality?","user")

        # inits patient simulation 
        logger.info("Initializing patient simulation of ID "+str(id)+" Vignette: "+ vignette + "testing.runPatientSimulation()")
        patientSimulation = await initPatientSimulation(user_context["user_id"],response,vignette)
        patientSimulation["user_id"] = id
        patientSimulation["user_context"] = user_context

        patientConvo = patientSimulation["patientConvo"]

        # gets doctor's first answer to patient's intro messages
        docAnswer = Conversation(langDocBack.initPatientConvo(patientConvo.lastMessage().content, user_context)["docConvo"])
        patientConvo.addMessage(UserMessage(docAnswer.lastMessage().content))
        patientSimulation["patientConvo"] = patientConvo
        user_context = patientSimulation["user_context"]
        while True:
                patientSimulation = generatePatientAnswer(patientSimulation)
                patientConvo = patientSimulation["patientConvo"]
                docreply = langDocBack.processResponse(patientConvo[-1].content, user_context)
                patientSimulation["patientConvo"].addMessage(UserMessage(docreply["docConvo"].lastMessage().content)) 
                # logger.info("Doctor: "+docReply[-1].content)
                # patientConvo.logger.info("\n\n TESTING COUNTMESSAGE FUNCTION")
                # logger.info("Count:"+str(patientConvo.countMessagesOfType(HumanMessage)))
    except Exception as e:
        logger.error(f"An error occurred: {e}")

In [6]:
#takes conversation and tests summary bot with it
def testSummaryBot(conversation):
    logger.info(langDocBack.summarizeData(conversation))

In [7]:
# returns example COnversation of key i
def exampleConversation(i):
    if i == 1:
        convo = Conversation()
        convo.addMessage("Hi, I am Dr. Who, how can I help you today.", "ai")
        convo.addMessage("Hi, I am Anna, 22 years old, and I have a headache.", "human")
        convo.addMessage("I see. Could you tell me more?", "AI")
        convo.addMessage("Well it started two days ago and it has been getting worse ever since. Now it is really bad and it's been affecting my sleep and concentration.", "human")
        convo.addMessage("Could you summarize the pain in more detail?", "ai")
        convo.addMessage("Yes certainly, it's a kind of diffuse, dull pain everywhere in my head and it's really bad. It gets worse when I lower my head.", "human")
        return convo

## Tests
### vignettes test

In [8]:
## initializes a demo conversation between a patient simulation and langdoc
def initVignettesDemo(vignettes):
    logger = logging.getLogger()
    logger.info("Initializing "+ str(len(vignettes))+ " vignettes. (testing.initVignettesDemo())")

    with Pool() as p:
        id_vignette_pairs = [(i+1, v) for i, v in enumerate(vignettes)]
        result = p.starmap_async(runPatientSimulation, id_vignette_pairs)
        result.get()
    # initializes patient simulations
    
    """
    logger.info("Initializing "+ str(vignettes.__len__())+ " vignettes. (testing.initVignettesDemo())")
    id = 1
    for v in vignettes:
        # runs patient simulation with vignette v and incrementally higher id 
        logger.info("Initializing patient simulation no."+str(id))
        coro = runPatientSimulation(id,  v)
        asyncio.create_task(coro)
        logger.info("TEST")
        id = id + 1 
    #loop.run_forever()
    """
       
def loadVignettes():
    f = open("vignettes.txt", "r")
    contents = f.read()
    vignettes = contents.split("===")
    return vignettes

vignettes = loadVignettes()
print("INIT")
initVignettesDemo(vignettes)
logger.info("INIT 2")


INFO:root:Initializing 2 vignettes. (testing.initVignettesDemo())


INIT


### Sending custom messages directly, WIP

In [ ]:
#discordFront.testing()
async def testing():
    await discordBack.sendCustomMessage("general","Hi")

asyncio.run(testing())

RuntimeError: asyncio.run() cannot be called from a running event loop

### Testing summary bot

In [ ]:
logger.info(testSummaryBot(exampleConversation(1)))

- Demographics:
    - Name: Anna
    - Age: 22 years old
    
- Symptoms:
    - Headache:
        - Duration: 2 days
        - Character: Diffuse, dull pain
        - Severity: Really bad
        - Aggravating factors: Lowering head
        - Associated symptoms: Sleep disturbance, difficulty concentrating
- Demographics:
    - Name: Anna
    - Age: 22 years old
    
- Symptoms:
    - Headache:
        - Duration: 2 days
        - Character: Diffuse, dull pain
        - Severity: Really bad
        - Aggravating factors: Lowering head
        - Associated symptoms: Sleep disturbance, difficulty concentrating
None


### demo convo
currently not working, minor adjustment of data types required

In [ ]:
await initDemoConversation()

Initializing LangDocAPI (langDocBack.initLangDocAPI())
Initializing LangDoc Agent (langDocBack.initDocAgent())
Doctor:Good morning! My name is Anna Johns, and I'll be assisting you today. How can I help you?(langDocBack.initDocAgent())


TypeError: list indices must be integers or slices, not str